In [191]:
!pip install einops

In [192]:
import einops
import torch

# Q1

Construct the tensor
`[[3, 4],
 [5, 6],
 [7, 8]]`
using only `torch.arange` and `einops.rearrange`.

In [193]:
tensor = torch.arange(3, 9)
einops.rearrange(tensor, '(a b) -> a b', a=3, b=2)

tensor([[3, 4],
        [5, 6],
        [7, 8]])

Now construct 
`[[1, 2, 3],
 [4, 5, 6]]`
and
`[[[1], [2], [3], [4], [5], [6]]]`.

In [194]:
tensor = torch.arange(1, 7)
einops.rearrange(tensor, '(a) -> 1 a 1')

tensor([[[1],
         [2],
         [3],
         [4],
         [5],
         [6]]])

# Q2

Get the average temperature by week from a vector of daily temperatures. The input is a 1d tensor whose length is a multiple of 7. For example:
`t.Tensor([71,72,70,75,71,72,70, 68,65,60,68,60,55,59, 75,80,85,80,78,72,83]) -> t.tensor([71.5714, 62.1429, 79.0000])`. 

In [195]:
daily_temperatures = torch.Tensor([71,72,70,75,71,72,70, 68,65,60,68,60,55,59, 75,80,85,80,78,72,83])
daily_temperatures_by_week = einops.rearrange(daily_temperatures, '(a b) -> a b', a=3, b=7)
daily_temperatures_by_week

tensor([[71., 72., 70., 75., 71., 72., 70.],
        [68., 65., 60., 68., 60., 55., 59.],
        [75., 80., 85., 80., 78., 72., 83.]])

Also turn the vector of temperatures into the difference between that temperature and the mean weekly temperature. For example:
`t.Tensor([71,72,70,75,71,72,70, 68,65,60,68,60,55,59, 75,80,85,80,78,72,83]) -> t.tensor([-0.5714,  0.4286, -1.5714,  3.4286, -0.5714,  0.4286, -1.5714,  5.8571, 2.8571, -2.1429,  5.8571, -2.1429, -7.1429, -3.1429, -4.0000,  1.0000,  6.0000,  1.0000, -1.0000, -7.0000,  4.0000])`.

In [196]:
weekly_average_temperatures = torch.mean(daily_temperatures_by_week, dim=1)
weekly_average_temperatures

tensor([71.5714, 62.1429, 79.0000])

In [197]:
daily_temperatures - einops.repeat(weekly_average_temperatures, '(a) -> (a 7)')

tensor([-0.5714,  0.4286, -1.5714,  3.4286, -0.5714,  0.4286, -1.5714,  5.8571,
         2.8571, -2.1429,  5.8571, -2.1429, -7.1429, -3.1429, -4.0000,  1.0000,
         6.0000,  1.0000, -1.0000, -7.0000,  4.0000])

Finally, turn the vector of daily temperatures into the difference between that temperature and the mean temperature for that week, scaled so its standard deviation is 1. Sometimes this is called “normalizing” the temperature by week. For example:
`t.Tensor([71,72,70,75,71,72,70, 68,65,60,68,60,55,59, 75,80,85,80,78,72,83]) -> t.Tensor([[-0.3326,  0.2494, -0.9146,  1.9954, -0.3326,  0.2494, -0.9146], [ 1.1839,  0.5775, -0.4331,  1.1839, -0.4331, -1.4438, -0.6353], [-0.8944,  0.2236,  1.3416,  0.2236, -0.2236, -1.5652,  0.8944]])`.

In [198]:
daily_temperatures_by_week_zero_mean = daily_temperatures_by_week - einops.repeat(weekly_average_temperatures, '(a) -> a 7')

standard_deviations = torch.std(daily_temperatures_by_week_zero_mean, dim=1)

daily_temperatures_by_week_normalized = daily_temperatures_by_week_zero_mean / einops.repeat(standard_deviations, '(a) -> a 7')

daily_temperatures_by_week_normalized

tensor([[-0.3326,  0.2494, -0.9146,  1.9954, -0.3326,  0.2494, -0.9146],
        [ 1.1839,  0.5775, -0.4331,  1.1839, -0.4331, -1.4438, -0.6353],
        [-0.8944,  0.2236,  1.3416,  0.2236, -0.2236, -1.5652,  0.8944]])

# Q3

Compute a batched dot product. Your function should take two tensors of the same shape `[i1, i2...in]` (which could be any shape!) and return one where the element at index `[i1, i2….in-1]` is the dot product of `a[i1,i2...in-1]` and `b[i1,i2...in-1]`. For example: `( t.Tensor([[1,1,0],[0,0,1]]), t.Tensor([[1,1,0],[1,1,0]]) ) -> t.Tensor([2., 0.])`.

In [199]:
def batched_dot_product(a, b):
  return torch.einsum('i ..., i ... -> i', a, b)

a = torch.Tensor([[1,1,0],[0,0,1]])
b = torch.Tensor([[1,1,0],[1,1,0]])

batched_dot_product(a, b)

tensor([2., 0.])

# Q4

Write a function that returns an identity matrix of size n x n. (Don’t use torch.eye or similar.)
`3 -> t.Tensor([[1, 0, 0]
    [0, 1, 0]
    [0, 0, 1]])`

hint: arange, rearrange, binary operations

In [200]:
def eye(n):
  return (torch.arange(n) == einops.rearrange(torch.arange(n), 'n -> n 1')).int()

eye(3)

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]], dtype=torch.int32)

# Q5

Let’s say there are K possible events, and each will happen with a probability in a k-length vector. Sample N times from this probability distribution.
For example, you could roll an uneven 6 sided die (numbered 0 to 5) with probabilities `[0.05, 0.1, 0.1, 0.2, 0.15, 0.4]` 5 times and get `[5, 1, 3, 5, 4]`.

Hint: torch.rand, torch.cumsum

In [201]:
def sample(distribution, n_samples):
  cdf = torch.cumsum(distribution, dim=0)
  repeat_cdf = einops.repeat(cdf, f"a -> {n_samples} a")

  samples = torch.rand(n_samples)
  repeat_samples = einops.repeat(samples, f"a -> a {len(distribution)}")

  return (repeat_samples <= repeat_cdf).int().argmax(dim=1)
  # return torch.multinomial(distribution, n_samples, replacement=True)

distribution = torch.tensor([0.05, 0.1, 0.1, 0.2, 0.15, 0.4])
sample(distribution, 5)

tensor([5, 2, 2, 3, 2])

# Q6

Suppose your classifier gives you an array of scores with shape `[n_inputs, n_classes]`, where the (i, j)-element represents how confidently the classifier thinks that example i belongs to class j. A higher number represents higher confidence, and the classifier’s prediction is the class with the highest score. 

Given such a score array and a vector of correct classes of length n_inputs, return the accuracy of the classifier: the fraction of inputs for which the maximum score corresponds to the correct class for that input.

`(tensor([[0.75, 0.5, 0.25], [0.1, 0.5, 0.4], [0.1, 0.7, 0.2]], tensor([0, 1, 0]))
->
tensor(0.6667)`

Hint: torch.argmax

In [202]:
def accuracy(scores, classes):
  return (scores.argmax(dim=1) == classes).float().mean()

scores = torch.tensor([[0.75, 0.5, 0.25], [0.1, 0.5, 0.4], [0.1, 0.7, 0.2]])
classes = torch.tensor([0, 1, 0])

accuracy(scores, classes)

tensor(0.6667)

# Q7

A shop sells K different items, charging `prices[k]` for item k (for k between 0 and K-1). Given a vector of item indices representing what different customers bought, calculate how much money the shop made.

`(prices=tensor([0.5, 1, 1.5, 2, 2.5], items=tensor([0, 0, 1, 1, 4, 3, 2])))
->
tensor(9.)`

Hint: pytorch advanced indexing: this is the same as numpy advanced indexing, explained here: https://numpy.org/doc/stable/reference/arrays.indexing.html#advanced-indexing, or torch.gather


In [203]:
def total_price(prices, items):
  return prices[items].sum()

prices = torch.tensor([0.5, 1, 1.5, 2, 2.5])
items = torch.tensor([0, 0, 1, 1, 4, 3, 2])

total_price(prices, items)

tensor(9.)